# 🔥 Fine-Tune E5-Large for Jassas Arabic Search

This notebook fine-tunes `intfloat/multilingual-e5-large` on your Arabic government documents.

**Requirements:**
- Runtime → Change runtime type → T4 GPU
- Your `training_pairs.jsonl` file

**Time:** ~1-2 hours for 5000 training pairs

## Step 1: Setup Environment

In [ ]:
# Install dependencies
!pip install -q sentence-transformers datasets accelerate

# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv

## Step 2: Upload Training Data

Upload your `training_pairs.jsonl` file (generated locally with `generate_training_data.py`)

In [ ]:
from google.colab import files

print("Upload your training_pairs.jsonl file:")
uploaded = files.upload()

# Check file
import json
with open('training_pairs.jsonl', 'r') as f:
    pairs = [json.loads(line) for line in f]
print(f"\n✓ Loaded {len(pairs)} training pairs")
print(f"\nExample:")
print(f"  Query: {pairs[0]['query']}")
print(f"  Positive: {pairs[0]['positive'][:80]}...")

## Step 3: Prepare Training Data

In [ ]:
import json
import random
from sentence_transformers import InputExample

# Load data
with open('training_pairs.jsonl', 'r') as f:
    pairs = [json.loads(line) for line in f]

# Shuffle and split
random.shuffle(pairs)
split_idx = int(len(pairs) * 0.9)
train_pairs = pairs[:split_idx]
eval_pairs = pairs[split_idx:]

print(f"Train: {len(train_pairs)}, Eval: {len(eval_pairs)}")

# Create training examples with E5 prefixes
# E5 REQUIRES "query: " and "passage: " prefixes!
all_positives = [p['positive'] for p in pairs]

train_examples = []
for pair in train_pairs:
    query = f"query: {pair['query']}"
    positive = f"passage: {pair['positive']}"
    
    # Random negative (in-batch negatives will help too)
    neg_candidates = [p for p in all_positives if p != pair['positive']]
    if neg_candidates:
        negative = f"passage: {random.choice(neg_candidates)}"
        train_examples.append(InputExample(texts=[query, positive, negative]))

print(f"\n✓ Created {len(train_examples)} training triplets")

## Step 4: Load Model

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = 'intfloat/multilingual-e5-large'

print(f"Loading {MODEL_NAME}...")
model = SentenceTransformer(MODEL_NAME)
print(f"✓ Model loaded")
print(f"  Embedding dimension: {model.get_sentence_embedding_dimension()}")

## Step 5: Configure Training

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import losses, evaluation

# Training config
BATCH_SIZE = 8      # Reduce to 4 if OOM
EPOCHS = 3          # 2-5 epochs is typical
LEARNING_RATE = 2e-5
WARMUP_RATIO = 0.1
OUTPUT_DIR = './jassas-e5-arabic'

# DataLoader
train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=BATCH_SIZE
)

# Loss: MultipleNegativesRankingLoss (InfoNCE)
# This is THE standard loss for embedding fine-tuning
train_loss = losses.MultipleNegativesRankingLoss(model)

# Evaluator
eval_examples = [
    InputExample(texts=[f"query: {p['query']}", f"passage: {p['positive']}"], label=1.0)
    for p in eval_pairs[:100]
]
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(
    eval_examples, name='arabic-gov'
)

# Calculate warmup
total_steps = len(train_dataloader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

print(f"Training config:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")

## Step 6: Train! 🚀

In [ ]:
# This will take 1-2 hours depending on data size
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path=OUTPUT_DIR,
    show_progress_bar=True,
    evaluation_steps=500,
    save_best_model=True,
)

print(f"\n✓ Training complete!")
print(f"  Model saved to: {OUTPUT_DIR}")

## Step 7: Test the Fine-Tuned Model

In [ ]:
import numpy as np

# Load fine-tuned model
finetuned = SentenceTransformer(OUTPUT_DIR)

# Test queries
test_queries = [
    "كم غرامة تأخير تجديد الجواز؟",
    "ما هي رسوم تجديد رخصة القيادة؟",
    "كيف أستخرج تأشيرة خروج وعودة؟"
]

# Test documents (mix of relevant and irrelevant)
test_docs = [
    "غرامة تأخير تجديد جواز السفر تبلغ 500 ريال عن كل سنة تأخير",
    "رسوم إصدار جواز السفر الجديد 300 ريال سعودي",
    "رسوم تجديد رخصة القيادة 40 ريال لمدة 5 سنوات",
    "يمكن استخراج تأشيرة الخروج والعودة عبر منصة أبشر",
    "الطقس اليوم مشمس في الرياض",  # Irrelevant
]

# Encode with prefixes
q_embeddings = finetuned.encode([f"query: {q}" for q in test_queries])
d_embeddings = finetuned.encode([f"passage: {d}" for d in test_docs])

# Calculate similarities
print("\n" + "="*60)
print("SIMILARITY MATRIX (Query vs Documents)")
print("="*60)

for i, query in enumerate(test_queries):
    print(f"\nQuery: {query}")
    scores = np.dot(q_embeddings[i], d_embeddings.T)
    ranked = sorted(zip(test_docs, scores), key=lambda x: -x[1])
    for doc, score in ranked[:3]:
        print(f"  [{score:.3f}] {doc[:60]}...")

## Step 8: Download Fine-Tuned Model

In [ ]:
# Zip the model folder
!zip -r jassas-e5-arabic.zip jassas-e5-arabic/

# Download
from google.colab import files
files.download('jassas-e5-arabic.zip')

print("\n" + "="*60)
print("NEXT STEPS:")
print("="*60)
print("1. Unzip jassas-e5-arabic.zip to your project's models/ folder")
print("2. Update src/tokenizer/vector.py:")
print("   MODEL_NAME = 'models/jassas-e5-arabic'")
print("3. Rebuild index: ./jassas build")
print("4. Test: ./jassas search 'غرامة الجواز'")

## (Optional) Push to HuggingFace Hub

In [ ]:
# Login to HuggingFace
from huggingface_hub import login
login()  # Enter your token

# Push model
finetuned.save_to_hub(
    repo_id="your-username/jassas-e5-arabic",
    private=True
)
print("✓ Pushed to HuggingFace Hub!")